# Google Drive Loading
It requires to mount the directory first
Please put the data in the same directory as below to avoid probems

In [0]:
import numpy as np
import pickle as pk
from google.colab import drive
import tensorflow as tf
import os
# drive.mount('/content/gdrive', force_remount=True)

In [21]:
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def motionLoad(classes, directory, window = 60):
  
  if directory.endswith("/"):
    dire = directory
  else:
    dire = directory+"/"
  file_list = os.listdir(dire)
#   print(file_list)
  file_list = [i for i in file_list if any(s in i for s in classes)]
#   print(file_list)
  instances = []
  labels = []
  #READ FILES
  for f in file_list:
    label = [0.,0.,0.]
    index = 0
    #CHECK CLASS
    for c, cla in enumerate(classes):
      if cla in f:
        label[c] = 1.0
    #OPEN FILE    
    with open(directory+f) as doc:
      lines = doc.readlines()
    end = 0
    #READ AND APPEND
    while index < len(lines):
      count = 0
      subin = []
      if index+window < len(lines):
        while count < window:
          count +=1
          out = lines[index].split("|m ")[1].split(" |class")[0].split(" ") # if you make changes look here
          out = [float(i) for i in out]
          subin.append(out)
          index +=1 # pay attention here!
        instances.append(np.array(subin))
        labels.append   (np.array(label))
      else:
        index = len(lines)
  return instances, labels
      
          
      

In [0]:
xv, yv = motionLoad(["Crouching", "Running", "Swing"], "/content/gdrive/My Drive/DLLab/Motion/VAL/", 60)
xt, yt = motionLoad(["Crouching", "Running", "Swing"], "/content/gdrive/My Drive/DLLab/Motion/TRAIN/", 60)
# motionLoad(["Crouching", "Running", "Swing"], "/content/gdrive/My Drive/DLLab/Motion/TEST/")

In [24]:
np.array(xv).shape # (BATCH, TIME, FEATURES)

(355, 60, 20)

# Task 2

## 1) Normalize the data-set

## 2) Build a LSTM architecture to classify this data-set

GIT: https://github.com/RafaelDrumond/PeekDB

Run for 25 epochs and report your results.
Classes used:  "Crouching", "Running", "Swing"

Use Actor 2 for validation, and actor 3 and onwards for training. Remember to use np.array to transform the list into numpy array.

Pay attention to:
  The size of the window (use 3 different sizes)

## 3) On the current state the loaded data-set is not optimal, you must improve the loading function (hint: the git points to a certain paper that might help with and augmentation)

## 4) BONUS: use early time-series to see if you can improve the scores

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
x = [[[1.,2.,3.,4.],[1.,2.,3.,4.],[0.,1.,2.,4.]]]
x = np.array(x)

In [ ]:
X      = tf.placeholder(tf.float32,(None, None, 4))
output = tf.keras.layers.GRU(2, return_sequences=True)(X)
output = tf.keras.layers.GRU(2, return_sequences=True)(output)
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(output,feed_dict={X:x}).shape)